In [2]:
import cv2
from face_detection import face
from keras.models import load_model
import numpy as np
import os
from embedding import emb
from retreive_pymongo_data import database
import warnings
warnings.filterwarnings("ignore")

label=None

people=sorted(os.listdir('people'))

abhi=None
def Create_labels():
    people=sorted(os.listdir('people'))
    students={}
    for i in people:
        students[(int(i[0])-1)]=i[1:]
    return students


people=Create_labels()
completed_label="Attendance is Completed"
#data=database()
e=emb()
fd=face()

In [3]:
a={0:0,1:0,2:0}

model=load_model('face_reco2.MODEL')
abhi=None

def test():
    test_run=cv2.imread('2.jpg',1)
    test_run=cv2.resize(test_run,(160,160))
    #test_run=np.rollaxis(test_run,2,0)
    test_run=test_run.astype('float')/255.0
    test_run=np.expand_dims(test_run,axis=0)
    test_run=e.calculate(test_run)
    test_run=np.expand_dims(test_run,axis=0)
    test_run=model.predict(test_run)[0]

color=(0, 255, 0) 
cap=cv2.VideoCapture(0)
ret=True

In [6]:
people

{0: 'bhavesh', 1: 'kushagra', 2: 'rishabh'}

In [7]:
a


{0: 0, 1: 0, 2: 0}

In [ ]:
color=(0, 255, 0) 
cap=cv2.VideoCapture(0)
ret=True
#test()
while ret:
    ret,frame=cap.read()
    frame=cv2.flip(frame,1)
    det,coor=fd.detectFace(frame)

    if(det is not None):
        for i in range(len(det)):
            detected=det[i]
            k=coor[i]
            f=detected
            detected=cv2.resize(detected,(160,160))
            #detected=np.rollaxis(detected,2,0)
            detected=detected.astype('float')/255.0
            detected=np.expand_dims(detected,axis=0)
            feed=e.calculate(detected)
            feed=np.expand_dims(feed,axis=0)
            prediction=model.predict(feed)[0]

            result=int(np.argmax(prediction))

            if(np.max(prediction)>.80):
                for i in people:
                    if(result==i):
                        label=people[i]
                        
                        if(a[i]<30):
                            #print("a")
                            a[i]=a[i]+1
                            abhi=i
            else:
                label='unknown'
            #data.update(label)


            
            if(int(a[abhi])==30):   
                cv2.putText(frame,completed_label,(k[0],k[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
                cv2.rectangle(frame,(k[0],k[1]),(k[0]+k[2],k[1]+k[3]),(0,255,0),3)
            else:
                cv2.putText(frame,label,(k[0],k[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
                cv2.rectangle(frame,(k[0],k[1]),(k[0]+k[2],k[1]+k[3]),(255,0,0),3)

            
            cv2.imshow('onlyFace',f)
    cv2.imshow('frame',frame)
    if(cv2.waitKey(1) & 0XFF==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()
data.export_csv()

In [184]:

from pymongo import MongoClient
import pandas as pd

class database:
    def __init__(self):
        self.client=MongoClient()
        self.db=self.client.students
        self.name=[]
        self.attendance=[]

    def update(self,name):
        self.db.Attendance.update_one({"name":name},{"$inc":{"Attendance":1}})


    def view(self):
        self.name=[]
        self.attendance=[]
        records=self.db.Attendance.find()
        j=0
        for i in records:
            j=j+1
            print(i["Name"])
            self.name.append(i["Name"])
            #self.Attendance.append(i[Attendance])
#         for i in range(j):
#             print(self.name[i],self.attendance[i])

    def export_csv(self,lecture):
        df=pd.DataFrame(columns=["Roll Number","Name","Attendance"])
        records=self.db.Attendance.find()
        for i in records:
            to_append=[i["Roll_number"],i["Name"],i["Attendance"]]
            a_series = pd.Series(to_append, index = df.columns)
            df=df.append(a_series,ignore_index=True)
        CSV_Name=lecture+"_Attendance.csv"
        df.to_csv(CSV_Name,index=True)

In [185]:
data=database()

In [186]:
data.export_csv("Hindi")

  Roll Number     Name  Attendance
0           1  bhavesh         2.0
  Roll Number      Name  Attendance
0           1   bhavesh         2.0
1           2  kushagra         0.0
  Roll Number      Name  Attendance
0           1   bhavesh         2.0
1           2  kushagra         0.0
2           3   rishabh         0.0
